In [ ]:
import random

from relativisticpy.interpreter.lexers.gr_lexer import GRLexer
from relativisticpy.interpreter.analyzers.semantic_analyser import SemanticAnalyzer
from relativisticpy.interpreter.parsers.gr_parser import GRParser
from relativisticpy.interpreter.interpreter import Interpreter

In [5]:
from dataclasses import dataclass, field
from typing import List

# Base classes and utilities
class ScopedState:
    def reset(self):
        pass

class Implementer:
    pass

@dataclass
class AstNode:
    node: str
    handler: str
    args: List["AstNode"] = field(default_factory=list)

class Instruction:
    """
    Base class for all instructions for the Intermediate Representation (IR) of the RelPy code.
    """
    def __init__(self, result: str):
        self.result = result

    def __str__(self):
        return f"{self.result} = Instruction"

# Instruction subclasses following your pattern
class IntegerInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.value = node.args[0]  # Assuming node.node contains the integer value as a string

    def __str__(self):
        return f"{self.result} = {self.value}"

class FloatInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.value = node.args[0]  # Assuming node.node contains the float value as a string

    def __str__(self):
        return f"{self.result} = {self.value}"

class AddInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.left = node.args[0]
        self.right = node.args[1]

    def __str__(self):
        left_temp = self.left
        right_temp = self.right
        return f"{self.result} = {left_temp} + {right_temp}"

class SubInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.left = node.args[0]
        self.right = node.args[1]

    def __str__(self):
        left_temp = self.left
        right_temp = self.right
        return f"{self.result} = {left_temp} - {right_temp}"

class MulInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.left = node.args[0]
        self.right = node.args[1]

    def __str__(self):
        left_temp = self.left
        right_temp = self.right
        return f"{self.result} = {left_temp} * {right_temp}"

class DivInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.numerator = node.args[0]
        self.denominator = node.args[1]

    def __str__(self):
        numerator_temp = self.numerator
        denominator_temp = self.denominator
        return f"{self.result} = {numerator_temp} / {denominator_temp}"

class PowInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.base = node.args[0]
        self.exponent = node.args[1]

    def __str__(self):
        base_temp = self.base
        exponent_temp = self.exponent
        return f"{self.result} = {base_temp} ** {exponent_temp}"

class NegInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.operand = node.args[0]

    def __str__(self):
        operand_temp = self.operand
        return f"{self.result} = -{operand_temp}"

class PosInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.operand = node.args[0]

    def __str__(self):
        operand_temp = self.operand
        return f"{self.result} = +{operand_temp}"

class SymbolInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.name = node.args[0]

    def __str__(self):
        return f"{self.result} = Symbol('{self.name}')"

class AssignmentInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.variable = node.args[0]
        self.value = node.args[1]

    def __str__(self):
        value_temp = self.value
        return f"{self.variable} = {value_temp}"

class FunctionCallInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.func_name = node.identifier
        self.args = node.args

    def __str__(self):
        args_temps = ', '.join(arg for arg in self.args)
        return f"CALL {self.func_name}, {args_temps}"

class EquationInstruction(Instruction):
    def __init__(self, result: str, node: AstNode):
        super().__init__(result)
        self.left = node.args[0]
        self.right = node.args[1]

    def __str__(self):
        left_temp = self.left
        right_temp = self.right
        return f"{self.result} = Eq({left_temp}, {right_temp})"

class ArrayInstruction(Instruction):
    def __init__(self, result: str, elements: List[AstNode]):
        super().__init__(result)
        self.elements = elements

    def __str__(self):
        array_string = f"ARRAY {self.result} \n   "
        return array_string + '\n   '.join(f"{self.result}[{idx}] = {instr}" for idx, instr in enumerate(self.elements))

class IR:
    def __init__(self):
        self.instructions = []

    def new_instruction(self, instruction: Instruction):
        self.instructions.append(instruction)

    def __str__(self):
        return '\n'.join(f"{idx}: {instr}" for idx, instr in enumerate(self.instructions, start=1))

    def __repr__(self):
        return self.__str__()

class MyImplementer(Implementer):
    def __init__(self, ir: IR):
        self.ir = ir
        self.temp_counter = 0
        self._state = ScopedState()
        self.symbol_table = {}  # For managing variable names

    def new_temp(self):
        temp_var = f"t{self.temp_counter}"
        self.temp_counter += 1
        return temp_var

    @property
    def state(self) -> ScopedState:
        return self._state

    @state.setter
    def state(self, state: ScopedState) -> None:
        self._state = state

    def clear(self, node: AstNode):
        self.state.reset()

    # Implementations for basic operations

    def int(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(IntegerInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def float(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(FloatInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def add(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(AddInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def sub(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(SubInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def mul(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(MulInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def div(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(DivInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def pow(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(PowInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def neg(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(NegInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def pos(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(PosInstruction(result_temp, node))
        node = result_temp
        return result_temp

    def symbol(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(SymbolInstruction(result_temp, node))
        return result_temp

    def assign(self, node: AstNode):
        self.ir.new_instruction(AssignmentInstruction(None, node))
        self.symbol_table[node.args[0]] = node.args[1]  # Update symbol table
        return node.args[0]

    # Implementations for function calls
    def func_call(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(FunctionCallInstruction(result_temp, node))
        return result_temp

    # Implementations for mathematical functions
    def sin(self, node: AstNode):
        return self.func_call(node)

    def cos(self, node: AstNode):
        return self.func_call(node)

    def tan(self, node: AstNode):
        return self.func_call(node)

    def exp(self, node: AstNode):
        return self.func_call(node)

    def ln(self, node: AstNode):
        return self.func_call(node)

    def sqrt(self, node: AstNode):
        return self.func_call(node)

    def absolute(self, node: AstNode):
        return self.func_call(node)

    # Implementations for special functions
    def diff(self, node: AstNode):
        return self.func_call(node)

    def integrate(self, node: AstNode):
        return self.func_call(node)

    def simplify(self, node: AstNode):
        return self.func_call(node)

    def expand(self, node: AstNode):
        return self.func_call(node)

    def lim(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(FunctionCallInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    def solve(self, node: AstNode):
        return self.func_call(node)

    # Implementation for equation
    def equation(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(EquationInstruction(result_temp, node))
        return result_temp

    # Implementation for constants
    def constant(self, node: AstNode):
        result_temp = self.new_temp()
        self.ir.new_instruction(FunctionCallInstruction(result_temp, node))
        node.temp_var = result_temp
        return result_temp

    # Implementations for arrays
    def array(self, node: AstNode):
        # Create an instruction for the array
        result_temp = self.new_temp()
        self.ir.new_instruction(ArrayInstruction(result_temp, node.args))
        return result_temp

    # Implementations for tensors and advanced constructs
    def tensor(self, node: AstNode):
        return self.func_call(node)

    def derivative(self, node: AstNode):
        return self.func_call(node)

    def integral(self, node: AstNode):
        return self.func_call(node)

    # Implement variable declarations if needed
    def var_decl(self, node: AstNode):
        self.symbol_table[node.node] = node.node
        node.temp_var = node.node
        return node.node

    # Implement print statement
    def print_(self, node: AstNode):
        self.ir.new_instruction(FunctionCallInstruction(None, node))
        return None


In [6]:
def build_ir(expr: str):
    tok = GRLexer(expr).tokenize()
    parser = GRParser(tok)
    res = parser.parse()
    sa = SemanticAnalyzer() 
    script = sa.analyse(res)
    interpreter = Interpreter(MyImplementer(IR()))
    interpreter.exe_script(script)
    return interpreter.implementor

In [8]:
build_ir("""
hello(1) + 2
""").ir

NotImplementedError: The object MyImplementer does not have the method 'symbolfunc()' required to 
                implement the Node: Call. 

In [15]:
a = [1, 2, 3, 4, 5]
import random
random.choice(a)

2

In [464]:
from relativisticpy.interpreter.lexers.base import TokenType

d = {
            "!": {"=": TokenType.NOTEQUAL},
            "%": {"=": TokenType.PERCENTEQUAL},
            "&": {"=": TokenType.AMPEREQUAL},
            "+": {"=": TokenType.PLUSEQUAL},
            ":": {"=": TokenType.COLONEQUAL},
            "=": {"=": TokenType.EQEQUAL},
            "@": {"=": TokenType.ATEQUAL},
            "^": {"=": TokenType.CIRCUMFLEXEQUAL},
            "|": {"=": TokenType.VBAREQUAL, "|": TokenType.VBARVBAR},
            "*": {
                "*": TokenType.DOUBLESTAR,
                "=": TokenType.STAREQUAL,
            },
            ">": {
                "=": TokenType.GREATEREQUAL,
                ">": TokenType.RIGHTSHIFT,
            },
            "/": {
                "/": TokenType.DOUBLESLASH,
                "=": TokenType.SLASHEQUAL,
            },
            "-": {
                "=": TokenType.MINEQUAL,
                ">": TokenType.RARROW,
            },
            "<": {
                "-": TokenType.LARROW,
                "=": TokenType.LESSEQUAL,
                "<": TokenType.LEFTSHIFTEQUAL,
                ">": TokenType.NOTEQUAL,
            },
        }

In [466]:
from relativisticpy.interpreter.lexers._l import nested_dict_get

nested_dict_get(d, "<")

TypeError: nested_dict_get() missing 1 required keyword-only argument: 'd'

In [517]:
from typing import List, ByteString, Optional


class SourceCode:
    def __init__(self, sc_bytes: ByteString):
        self.__sc_bytes = sc_bytes

    @property
    def as_byte_list(self) -> List[int]:
        """Return the source code as a list of integer UTF-8 byte values."""
        return list(self.__sc_bytes)

    @property
    def as_unicode(self) -> List[int]:
        return list(self.__sc_bytes)

    @property
    def as_string(self) -> str:
        """Return the source code as a decoded UTF-8 string."""
        return self.__sc_bytes.decode('utf-8')

    @property
    def as_byte_string(self) -> ByteString:
        """Return the source code as raw UTF-8 encoded bytes."""
        return self.__sc_bytes

    @classmethod
    def from_string(cls, s: str):
        """Create a SourceCode object from a string."""
        return cls(s.encode('utf-8'))

    @classmethod
    def from_file(cls, file_path: str):
        """Create a SourceCode object from a UTF-8 encoded file."""
        with open(file_path, 'r', encoding='utf-8') as file:
            source_code = file.read()
        return cls(source_code.encode('utf-8'))

    @classmethod
    def from_stream(cls, stream):
        """Create a SourceCode object from a stream (e.g., a file-like object)."""
        source_code = stream.read()
        return cls(source_code.encode('utf-8'))

    @classmethod
    def from_bytes(cls, b: bytes):
        """Create a SourceCode object from raw bytes."""
        return cls(b)

    def __add__(self, other):
        """Concatenate two SourceCode objects."""
        return SourceCode.from_string(self.as_string + other.as_string)

    def __len__(self) -> int:
        """Return the length of the source code in characters."""
        return len(self.as_string)

    def __iter__(self):
        """Iterate over the integer representation of each UTF-8 character."""
        return iter(self.as_byte_list)

    def __enter__(self):
        """Enter a context manager."""
        # Return self to allow access to the SourceCode object in the `with` block.
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """Exit the context manager."""
        # No resources to clean up, but this ensures compatibility with `with`.
        pass

    def char_at(self, position: int) -> str:
        """
        Get the character at a specific position in the source code.
        :param position: The index of the character (0-based).
        :return: The character at the specified position.
        """
        if position < 0 or position >= len(self):
            raise IndexError(f"Position {position} is out of range for source code of length {len(self)}.")
        return self.as_string[position]

    def peek(self, position: int, count: int = 1) -> str:
        """
        Peek ahead to view characters starting at a specific position.
        :param position: The starting position (0-based).
        :param count: The number of characters to preview.
        :return: A substring of the specified length starting from the position.
        """
        if position < 0 or position >= len(self):
            raise IndexError(f"Position {position} is out of range for source code of length {len(self)}.")
        return self.as_string[position:position + count]

    def slice(self, start: int, end: Optional[int] = None) -> str:
        """
        Get a substring of the source code.
        :param start: The starting index (0-based, inclusive).
        :param end: The ending index (0-based, exclusive). If None, goes to the end of the string.
        :return: The sliced substring.
        """
        if start < 0 or start >= len(self) or (end is not None and (end < start or end > len(self))):
            raise IndexError(f"Invalid slice range {start}:{end} for source code of length {len(self)}.")
        return self.as_string[start:end]

    def is_empty(self) -> bool:
        """Check if the source code is empty."""
        return len(self.__sc_bytes) == 0

    def remaining_bytes(self, start: int) -> List[int]:
        """
        Get the remaining bytes in the source code starting from a given position.
        :param start: The starting position.
        :return: A list of remaining byte values.
        """
        if start < 0 or start >= len(self.as_byte_list):
            raise IndexError(f"Position {start} is out of range for source code of length {len(self)}.")
        return self.as_byte_list[start:]

In [610]:


class CharacterSets:
    """Defines logical groups of characters as class-level attributes with utility methods."""

    # Single-character definitions
    NEWLINE = {ord('\n')}
    COMMENT = {ord('#')}

    # Multi-character group definitions
    WHITESPACE = {ord(' '), ord('\t')}
    DELIMITERS = {ord(';'), ord('\n')}
    BRACKETS = {ord('['), ord(']')}
    PARENTHESES = {ord('('), ord(')')}
    BRACES = {ord('{'), ord('}')}

    ARITHMETIC_OPERATORS = {ord('+'), ord('-'), ord('*'), ord('/'), ord('%')}
    COMPARISON_OPERATORS = {ord('>'), ord('<'), ord('=')}
    LOGICAL_OPERATORS = {ord('&'), ord('|'), ord('~'), ord('^'), ord('!')}
    ASSIGNMENT_OPERATORS = {ord('=')}

    STRING_DELIMITERS = {ord("'"), ord('"')}
    NUMERIC_CHARACTERS = {ord(char) for char in "0123456789.e"}
    LOWERCASE = {ord(char) for char in "abcdefghijklmnopqrstuvwxyz"}
    UPPERCASE = {ord(char) for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZ"}
    IDENTIFIERS = LOWERCASE | UPPERCASE | NUMERIC_CHARACTERS | {ord('_')}
    SPECIAL = {ord('@'), ord(':')}

    @classmethod
    def all(cls) -> set:
        """Return a set of all defined characters."""
        return (
            cls.NEWLINE | 
            cls.COMMENT |
            cls.WHITESPACE |
            cls.DELIMITERS |
            cls.BRACKETS |
            cls.PARENTHESES |
            cls.BRACES |
            cls.ARITHMETIC_OPERATORS |
            cls.COMPARISON_OPERATORS |
            cls.LOGICAL_OPERATORS |
            cls.ASSIGNMENT_OPERATORS |
            cls.STRING_DELIMITERS |
            cls.NUMERIC_CHARACTERS |
            cls.LOWERCASE |
            cls.UPPERCASE |
            cls.IDENTIFIERS |
            cls.SPECIAL
        )

    @classmethod
    def as_strings(cls, group: set) -> set:
        """
        Convert a set of ord values into a set of characters.
        :param group: The group of ord values.
        :return: A set of string characters.
        """
        return {chr(o) for o in group}

    @classmethod
    def as_ord_list(cls, group: set) -> list:
        """
        Convert a set of ord values into a sorted list of ord values.
        :param group: The group of ord values.
        :return: A sorted list of ord values.
        """
        return sorted(group)

    @classmethod
    def validate_source_code(cls, source_code: str) -> set:
        """
        Validate a source code string, returning invalid characters.
        :param source_code: The source code to validate.
        :return: A set of invalid characters (as strings).
        """
        allowed_chars = cls.all()
        invalid_chars = {char for char in source_code if ord(char) not in allowed_chars}
        return invalid_chars

    @classmethod
    def is_valid_char(cls, char: str) -> bool:
        """
        Check if a single character is valid.
        :param char: The character to check.
        :return: True if the character is valid, False otherwise.
        """
        return ord(char) in cls.all()
    
    


CharSet.as_strings(CharSet.LOWERCASE)

{'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [533]:
sc = SourceCode.from_string("Hello, World!")
for char in sc:
    print(char in CharSet.as_strings(CharSet.UPPERCASE))

False
False
False
False
False
False
True
False
False
False
False
False
False


In [529]:
' ' in CharSet.WHITESPACE

False

In [537]:
ord('a') - ord('A')

32

In [538]:
def fast_uppercase(s: str) -> str:
    return chr(ord(s) - 32)


In [545]:
chr(913)

'Α'

In [546]:
ord('A')

65

In [550]:
chr(128061)

'🐽'

In [2]:
from enum import Enum, auto


class EnumObj(Enum):
    HELLO = auto()

<EnumObj.HELLO: 1>

0.0

In [5]:
from relativisticpy.interpreter.lexers.operators import SINGLE
from relativisticpy.interpreter.lexers.char_set import CharSet

from enum import Enum, auto
from typing import List
from relativisticpy.interpreter.shared.iterator import Iterator

from dataclasses import dataclass


def enum__contains(cls):
    """Enum.has_value(value) -> True or False -> if Enum contains that value."""

    @classmethod
    def has_value(cls, value):
        return any(value == item.value for item in cls)

    cls.has_value = has_value
    return cls


class TokenType(Enum):
    """An enumeration of token types used by a the lexer to genrate tokens."""

    NONE = auto()
    NEWLINE = auto()
    BACKSLASH = auto()
    DOUBLEBACKSLASH = auto()

    PLUS = auto(), "+"
    MINUS = auto(), "-"
    STAR = auto(), "*"
    SLASH = auto(), "/"
    EQUAL = auto(), "="
    LPAR = auto(), "("
    RPAR = auto(), ")"
    COMMA = auto(), ","
    LSQB = auto(), "["
    RSQB = auto(), "]"
    DOT = auto(), "."
    CIRCUMFLEX = auto(), "^"
    UNDER = auto(), "_"
    EXCLAMATION = auto(), "!"
    PERCENT = auto(), "%"
    AMPER = auto(), "&"
    COLON = auto(), ":"
    SEMI = auto(), ";"
    LESS = auto(), "<"
    GREATER = auto(), ">"
    AT = auto(), "@"
    LBRACE = auto(), "{"
    RBRACE = auto(), "}"
    VBAR = auto(), "|"
    TILDE = auto(), "~"
    APOSTROPHE = auto(), "'"
    
    NOTEQUAL = auto(), " '!=' or '<>' "
    PERCENTEQUAL = auto(), "%="
    AMPEREQUAL = auto(), "&="
    AMPERAMPER = auto(), "&&"
    DOUBLESTAR = auto(), "**"
    STAREQUAL = auto(), "*="
    PLUSEQUAL = auto(), "+="
    MINEQUAL = auto(), "-="
    RARROW = auto(), "->"
    LARROW = auto(), "<-"
    DOUBLESLASH = auto(), "//"
    SLASHEQUAL = auto(), "/="
    COLONEQUAL = auto(), ":="
    LEFTSHIFT = auto(), "<<"
    LESSEQUAL = auto(), "<="
    EQEQUAL = auto(), "=="
    GREATEREQUAL = auto(), ">="
    RIGHTSHIFT = auto(), ">>"
    ATEQUAL = auto(), "@="
    CIRCUMFLEXEQUAL = auto(), "^="
    VBAREQUAL = auto(), "|="
    VBARVBAR = auto(), "||"

    # Tripple Character Tokens
    DOUBLESTAREQUAL = auto(), "**="
    ELLIPSIS = auto(), "..."
    DOUBLESLASHEQUAL = auto(), "//="
    LEFTSHIFTEQUAL = auto(), "<<="
    RIGHTSHIFTEQUAL = auto(), ">>="

    ID = auto(), "A variable which references data."
    INT = auto(), "An Integer."
    BOOL = auto(), "A boolean."
    FLOAT = auto(), "A floating point number."
    SYMBOL = auto(), "A mathematical variable and/or symbol."
    STRING = auto(), "A string object."

    NOT = auto(), "not"
    AND = auto(), "and"
    OR = auto(), "or"
    PRINT = auto(), "print"
    INFINITESIMAL = auto(), "An infinitesimal d"
    PI = auto(), "The numerical constant pi"
    IM = auto(), "Imaginary number 'i'"
    E = auto(), "Euler's constant 'e' stored as numerical value."
    INFINITY = auto(), " 'infty' or 'oo' or utf-8 infinity 8734."
    CONSTANT = auto(), "The 'const' key word."

    SUM = auto(), "sum"
    LIMIT = auto(), "lim"
    FRAC = auto(), "frac"
    BEGIN = auto(), "begin"
    END = auto(), "end"
    DOSUM = auto(), "dosum"
    TO = auto(), "to"
    RIGHTARROW = auto(), "rightarrow"
    LEFTARROW = auto(), "leftarrow"
    PROD = auto(), "prod"
    DOPROD = auto(), "A command key word to perform the product: 'doprod'"
    EQUIVALENT = auto(), "equiv"
    PDERIVATIVE = auto(), "pdv"
    DERIVATIVE = auto(), "dv"
    INTEGRATE = auto(), "int"
    PARTIAL = auto(), "partial"
    SQRT = auto(), "sqrt"

    ##########################################
    ############   Greek Symbols  ############ 
    ##########################################
    alpha = auto(), "Greek symbol for alpha, lower case."
    Alpha = auto(), "Greek symbol for Alpha, upper case."
    beta = auto(), "Greek symbol for beta, lower case."
    Beta = auto(), "Greek symbol for Beta, upper case."
    gamma = auto(), "Greek symbol for gamma, lower case."
    Gamma = auto(), "Greek symbol for Gamma, upper case."
    delta = auto(), "Greek symbol for delta, lower case."
    Delta = auto(), "Greek symbol for Delta, upper case."
    epsilon = auto(), "Greek symbol for epsilon, lower case."
    Epsilon = auto(), "Greek symbol for Epsilon, upper case."
    zeta = auto(), "Greek symbol for zeta, lower case."
    Zeta = auto(), "Greek symbol for Zeta, upper case."
    eta = auto(), "Greek symbol for eta, lower case."
    Eta = auto(), "Greek symbol for Eta, upper case."
    theta = auto(), "Greek symbol for theta, lower case."
    Theta = auto(), "Greek symbol for Theta, upper case."
    iota = auto(), "Greek symbol for iota, lower case."
    Iota = auto(), "Greek symbol for Iota, upper case."
    kappa = auto(), "Greek symbol for kappa, lower case."
    Kappa = auto(), "Greek symbol for Kappa, upper case."
    lambda_ = auto(), "Greek symbol for lambda, lower case. Node: lambda is a reserved python keyword so we add underscore."
    Lambda = auto(), "Greek symbol for Lambda, upper case."
    mu = auto(), "Greek symbol for mu, lower case."
    Mu = auto(), "Greek symbol for Mu, upper case."
    nu = auto(), "Greek symbol for nu, lower case."
    Nu = auto(), "Greek symbol for Nu, upper case."
    xi = auto(), "Greek symbol for xi, lower case."
    Xi = auto(), "Greek symbol for Xi, upper case."
    omicron = auto(), "Greek symbol for omicron, lower case."
    Omicron = auto(), "Greek symbol for Omicron, upper case."
    pi = auto(), "Greek symbol for pi, lower case."
    Pi = auto(), "Greek symbol for Pi, upper case."
    rho = auto(), "Greek symbol for rho, lower case."
    Rho = auto(), "Greek symbol for Rho, upper case."
    sigma = auto(), "Greek symbol for sigma, lower case."
    Sigma = auto(), "Greek symbol for Sigma, upper case."
    tau = auto(), "Greek symbol for tau, lower case."
    Tau = auto(), "Greek symbol for Tau, upper case."
    upsilon = auto(), "Greek symbol for upsilon, lower case."
    Upsilon = auto(), "Greek symbol for Upsilon, upper case."
    phi = auto(), "Greek symbol for phi, lower case."
    Phi = auto(), "Greek symbol for Phi, upper case."
    chi = auto(), "Greek symbol for chi, lower case."
    Chi = auto(), "Greek symbol for Chi, upper case."
    psi = auto(), "Greek symbol for psi, lower case."
    Psi = auto(), "Greek symbol for Psi, upper case."
    omega = auto(), "Greek symbol for omega, lower case."
    Omega = auto(), "Greek symbol for Omega, upper case."


@dataclass
class Token:
    type: TokenType = None
    value: List[int] = None
    position: int = None


@dataclass
class LexerResult:
    code: SourceCode
    tokens: List[Token]


class Lexer:
    
    @staticmethod
    def new_iterator(source_code: SourceCode) -> Iterator[int]:
        return Iterator(source_code.as_byte_list)

    @classmethod
    def tokenize_file(cls, path: str):
        lexer = cls()
        source_code = SourceCode.from_file(path)
        return lexer.tokenize(source_code)

    def tokenize(self, source_code: SourceCode) -> LexerResult:
        self.char_iter = self.new_iterator(source_code)
        self.tokens = []
        
        while self.char_iter.current() is not None:
            
            if self.char_iter.current() in CharSet.WHITESPACE:
                self.char_iter.advance()

            elif self.char_iter.current() in CharSet.COMMENT:
                self._skip_comment()

            elif self.char_iter.current() in CharSet.DELIMITERS:
                self.char_iter.advance()
                token = Token(
                    TokenType.NEWLINE,
                    self.char_iter.current(),
                    self.char_iter.current_item_location,
                )
                self.tokens.append(token)

            elif self.char_iter.current() == CharSet.BACKSLASH:
                self._build_latex()

            elif self.char_iter.current() in CharSet.LOWERCASE | CharSet.UPPERCASE:
                self._identifiers()

            elif self.char_iter.current() in CharSet.DIGITS:
                self._build_number()

            elif self.char_iter.current() in CharSet.OPERATORS:
                self._operation()

            else:
                raise Exception(f"Illegal Character '{self.char_iter.current()}'")

        return LexerResult(self.source_code, self.tokens)

    def _build_number(self):
        number = []
        while self.char_iter.current() is not None and (self.char_iter.current() in CharSet.NUMERIC_CHARACTERS):
            number.append(self.char_iter.current())
            self.char_iter.advance()
        count = number.count(".")
        if count == 0:
            token = Token(
                TokenType.INT,
                number,
                self.char_iter.current_item_location
            )
            self.tokens.append(token)
        elif count == 1:
            token = Token(
                TokenType.FLOAT,
                number,
                self.char_iter.current_item_location
            )
            self.tokens.append(token)
        else:
            raise Exception(f"Illegal Character '{number}'")

    def _operation(self):
        operations = []
        doubles_dic = TokenType.DOUBLES()
        triples_dic = TokenType.TRIPPLES()
        while self.char_iter.current() is not None and self.char_iter.current() in TokenType.SINGLES():
            operations.append(self.char_iter.current())
            self.char_iter.advance()
        
        
        if len(operations) == 0:
            raise Exception("No operation found.")
        
        if len(operations) > 3:
            raise Exception("Too many operations.")
        
        if len(operations) == 1:
            op = operations[0]
            tok_type = SINGLE.get(op)
            token = Token(
                tok_type,
                [op],
                self.char_iter.current_item_location
            )
            self.tokens.append(token)
            return
        
        i = 0
        while i < len(ops):
            if (
                    i + 2 < len(ops)
                    and ops[i] in triples_dic
                    and self.token_provider.tripple_match_exists(ops[i], ops[i + 1], ops[i + 2])
            ):
                self.token_provider.new_tripple_operation_token(
                    ops[i],
                    ops[i + 1],
                    ops[i + 2],
                    end_pos=self.current_pos(),
                )
                i += 3
            elif (
                    i + 1 < len(ops)
                    and ops[i] in doubles_dic
                    and self.token_provider.double_match_exists(ops[i], ops[i + 1])
            ):
                self.token_provider.new_double_operation_token(
                    ops[i], ops[i + 1], end_pos=self.current_pos()
                )
                i += 2
            else:
                self.token_provider.new_single_operation_token(
                    ops[i], end_pos=self.current_pos()
                )
                i += 1

    # Here we are building: function - object - tensor
    def _identifiers(self):
        obj = ""
        while self.char_iter.current() is not None and self.char_iter.current() in Characters.IDS.value:
            obj += self.char_iter.current()

            if self.peek_char(1, None) == "_" and self.peek_char(2, None) in Characters.IDS.value:
                self.char_iter.advance()
                obj += self.char_iter.current()

            self.char_iter.advance()

        if obj in TokenType.Keywords():
            self.token_provider.new_token(TokenType.Keywords()[obj], obj, end_pos=self.current_pos())
        else:
            self.token_provider.new_token(TokenType.ID, obj, end_pos=self.current_pos())

    def _build_latex(self):
        obj = ""
        self.char_iter.advance()

        while self.char_iter.current() is not None and self.char_iter.current() in Characters.LETTERS.value:
            obj += self.char_iter.current()
            self.char_iter.advance()

        if self.char_iter.current() == TokenType.BACKSLASH.value:
            self.token_provider.new_token(
                TokenType.DOUBLEBACKSLASH,
                obj,
                end_pos=self.current_pos(),
            )
            self.char_iter.advance()
        elif obj not in TokenType.LaTeX():
            raise ValueError("TOKENIZER LEVEL ERROR: The latex keyword entered is not supported.")

        if obj in TokenType.LaTeX():
            self.token_provider.new_token(
                TokenType.LaTeX()[obj],
                obj,
                end_pos=self.current_pos(),
            )

    def _skip_comment(self):
        self.char_iter.advance()

        while self.char_iter.current() != "\n":
            self.char_iter.advance()

        self.char_iter.advance()


NameError: name 'SourceCode' is not defined

In [627]:
10 in CharSet.DELIMITERS

True

In [607]:
iterator.original

'hello'

In [7]:
{1} | {2}

{1, 2}

In [3]:
chr(8734)

'∞'

In [20]:
def binary_search(arr, x):
    low = 0
    high = len(arr) - 1
    mid = 0

    while low <= high:

        mid = (high + low) // 2

        if arr[mid] < x:
            low = mid + 1

        elif arr[mid] > x:
            high = mid - 1

        else:
            return mid

    return -1



In [21]:
arr = [2, 3, 4, 10, 40]
x = 10

result = binary_search(arr, x)

In [22]:
result

3